In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# -------------------------------
# Configuration and Setup
# -------------------------------

# Set the device: use CUDA if available otherwise CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

val_transform = transforms.Compose([
    transforms.Resize([256, 256]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
])

def load_model(model_path):
    model = EfficientNet.from_pretrained('efficientnet-b4')
    model._fc = nn.Linear(model._fc.in_features, 2)
    state_dict = torch.load(model_path, map_location=device)
    if next(iter(state_dict)).startswith('module.'):
        state_dict = {k.partition('module.')[2]: v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return model

# -------------------------------
# Functions for Image Processing and Inference
# -------------------------------

def preprocess_image(image_path):
    """
    Preprocess a single image: open, convert to RGB, apply transformation, and
    move the tensor to the specified device.
    """
    try:
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None
    image = val_transform(image)
    image = image.unsqueeze(0).to(device)
    return image

def predict_image(model, image_tensor):
    """
    Perform inference on a preprocessed image and return the class 1 probability.
    """
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        class_1_probability = probabilities[:, 1].item()  # Probability for the positive class
    return class_1_probability

# -------------------------------
# Main Batch Processing
# -------------------------------

def process_directory(model, image_dir, output_file):
    """
    Processes all images in the specified directory, prints results, and saves to a file.
    """
    image_filenames = os.listdir(image_dir)
    results = []

    # Iterate over each image in the directory.
    for filename in image_filenames:
        image_path = os.path.join(image_dir, filename)
        image_tensor = preprocess_image(image_path)
        if image_tensor is None:
            # Skip images that failed to load or process.
            continue
        probability = predict_image(model, image_tensor)
        results.append((filename, probability))
        print(f"Image: {filename}, Probability: {probability:.4f}")

    # Write the results to the designated text file.
    with open(output_file, "w") as file:
        for filename, probability in results:
            file.write(f"Image: {filename}, Probability HCQ toxicity: {probability:.4f}\n")
    print(f"\nResults saved to {output_file}")

# -------------------------------
# Running the Script
# -------------------------------

if __name__ == "__main__":
    # Specify paths (change these to your actual paths or consider using arguments)
    model_path = 'hcquery_model.pt'
    image_dir = "" 
    output_file = ""

    model = load_model(model_path)

    process_directory(model, image_dir, output_file)